In [3]:
import numpy as np
import re
import nltk
import os
from random import shuffle
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.models import model_from_json
from newspaper import Article


Using TensorFlow backend.
/home/rashi/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
class lstm_model:
    def transform_keywords(self, file_name):
        inf_file = open(file_name)
        data = list()
        for one_news in inf_file.readlines():
            single = one_news.strip().split(',')
            mapping = list()
            for one_keyword in single:
                mapping.append(one_hot(one_keyword, 7000)[0])
            data.append(mapping)
        print(4,data)
        return data

    def transform_titles(self, file_name):
        inf_file = open(file_name)
        data = list()
        for one_news in inf_file.readlines():
            single = nltk.word_tokenize(self.clean_sentence(one_news))
            mapping = list()
            for one_keyword in single:
                mapping.append(one_hot(one_keyword, 7000)[0])
            data.append(mapping)
        print(5,data)
        return data

    def clean_sentence(self,s):
        print("3",s)
        data = ""
        for item in s:
            item.lower().strip()
        #c = s.lower().strip()
        return re.sub('[^a-z ]', '', " ".join(s))

    '''
    :param
        type: 0 indicates using the keywords from the content
              1 indicates using the titles
    '''
    def save_model(self, fake_file, real_file, type, model_name, unit_size = 10):
        batch_size = 10
        if type == 0:
            fake_data = self.transform_keywords(fake_file)
            real_data = self.transform_keywords(real_file)
        else:
            fake_data = self.transform_titles(fake_file)
            real_data = self.transform_titles(real_file)
            batch_size = 32
        labels = list()
        max_len = 0
        for i in fake_data:
            labels.append(0)
        for i in real_data:
            labels.append(1)
        data=fake_data
        for r in fake_data:
            if max_len < len(r):
                max_len = len(r)
        for r in real_data:
            if max_len < len(r):
                max_len = len(r)
            data.append(r)
        for d in data:
            cur_len = len(d)
            while cur_len < max_len:
                d.append(0)
                cur_len = cur_len+1

        #shuffle the given data
        index_shuf = list(range(len(data)))
        shuffle(index_shuf)
        data_shuffled = list()
        label_shuffled = list()
        for i in index_shuf:
            data_shuffled.append(data[i])
            label_shuffled.append(labels[i])
        model = Sequential()
        model.add(Embedding(7000, 256, dropout=0.2))
        model.add(LSTM(16, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        model.compile(loss='binary_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])

        model.fit(data_shuffled, label_shuffled,
                  nb_epoch=10,
                  batch_size=batch_size,
                  shuffle=False)
        # serialize model to JSON
        model_json = model.to_json()
        with open(model_name, "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model.save_weights("model.h5")
        print("Saved model to disk")

    def reload_model(self, file_name):
        json_file = open(file_name, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        # load weights into new model
        loaded_model.load_weights("model.h5")
        print("Loaded model from disk")
        return loaded_model

    #max_len for keywords: , max_len_for_titles:
    def format_testcase(self,string, type, max_len):
        #titles
        single = list()
        if type == 0:
            single = nltk.word_tokenize(self.clean_sentence(string))
        #keywords
        else:
            single = string
        mapping = list()
        for one_keyword in single:
            mapping.append(one_hot(one_keyword, 7000)[0])
        while len(mapping) < max_len:
            mapping.append(0)
        data = list()
        data.append(mapping)
        print(data)
        return mapping


In [9]:
if __name__ == "__main__":
    #Task on the content keywords
    #save_model("./fakenews_keywords.csv","./realnews_keywords.csv",0,"model_keywords.json")
    #Task on the titles
    #save_model("./data/titles/fake_news_training.txt", "./data/titles/real_news_training.txt",1,"model_titles.json" )
    #Redload a save model:
    #Extract keywords from the web
    article = Article('http://www.bbc.com/news')
    article.download()
    article.parse()
    article.nlp()
    print(article.keywords)
    print("article keywords have been printed")
    mod = lstm_model()
    model = mod.save_model('data/fake_news_training.txt','data/real_news_training.txt',0,'model_titles.json')

    '''parse the input from web front end, say the keywords/titles are raw strings
    TODO: 1) Preprocess the string such that only words count
          2) Form the corresponding matrix by calling the function like:
            (max_len is determined by the training model)
            format_testcase(string, type=1 for keywords, 0 for title, max_len = 19 for keywords, =39 for titles):'''


    #model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    X = mod.format_testcase(article.keywords, 0,9)
    #result = model.predict(X) # change 19 to the corresponding  max_len(19 or 39)
    print(result)

['bbc', 'crashthe', 'route', 'crashed', 'inexplicable', 'operator', 'killing', 'usual', 'seaplane', 'sydney', 'path']
article keywords have been printed
4 [[5668], [2456], [5796], [117, 1950], [2855, 3630], [746], [2276], [4959], [4354], [780], [1709], [4673], [4731], [5726, 749], [3436], [2332, 4673], [2638], [6289, 4967], [6582, 6123], [3760], [5857, 6007], [5000], [4673], [951], [6526], [671], [4142, 5419], [107], [5000], [948], [3891], [4967], [6221], [6943, 4846], [3284], [5360], [3279], [4977], [1035], [4673], [5727], [1679], [3388, 376], [5472], [5820], [5636, 819], [3379, 2402, 3581], [3145, 5894], [2281], [5318, 395], [1560], [4411], [749, 4932, 4951], [2685], [2221], [4015], [3359, 547], [1744], [190], [6239], [10, 3880], [1051, 4558], [4015, 4293], [5358], [4372, 4691], [6815], [4160], [1561], [12], [6003, 2644], [641], [3760], [1999], [1806], [6864, 4678], [5433], [4673], [4411], [4902], [5145], [1292], [1806], [4673], [1110], [5416], [3379], [4321], [107], [2124], [3247], 

/home/rashi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
/home/rashi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:78: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(16, dropout=0.2, recurrent_dropout=0.2)`
/home/rashi/anaconda3/lib/python3.6/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
144/144 [==============================] - 1s 6ms/step - loss: 0.5871 - acc: 0.7986
Epoch 2/10
144/144 [==============================] - 1s 3ms/step - loss: 0.5117 - acc: 0.7986
Epoch 3/10
144/144 [==============================] - 1s 4ms/step - loss: 0.4978 - acc: 0.7986
Epoch 4/10
144/144 [==============================] - 1s 3ms/step - loss: 0.4877 - acc: 0.7986
Epoch 5/10
144/144 [==============================] - 1s 4ms/step - loss: 0.4799 - acc: 0.7986
Epoch 6/10
144/144 [==============================] - 0s 3ms/step - loss: 0.4583 - acc: 0.7986
Epoch 7/10
144/144 [==============================] - 0s 3ms/step - loss: 0.4347 - acc: 0.7986
Epoch 8/10
144/144 [==============================] - 1s 4ms/step - loss: 0.4017 - acc: 0.7986
Epoch 9/10
144/144 [==============================] - 0s 3ms/step - loss: 0.3473 - acc: 0.8194
Epoch 10/10
144/144 [==============================] - 1s 4ms/step - loss: 0.2735 - acc: 0.8681
Saved model to disk
3 ['bbc', 'crashthe', 'route'

AttributeError: 'NoneType' object has no attribute 'predict'